<a href="https://colab.research.google.com/github/enderunknown222/MA/blob/Ver0.2/MA4550_Project_ver0.2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import io
from sklearn.multioutput import MultiOutputRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
pd.options.mode.chained_assignment = None
from google.colab import files
data = pd.read_csv('https://raw.githubusercontent.com/enderunknown222/MA/Ver0.1/data.csv')

In [3]:
#test data size
x=190

Part 1. Predict without any stats

In [ ]:
#Change name to ID
results = data.iloc[:,1:7]
Name=['Arsenal','Aston Villa','Brighton','Burnley','Chelsea','Crystal Palace','Everton','Fulham','Leeds United','Leicester City','Liverpool','Manchester City','Manchester Utd','Newcastle Utd','Sheffield Utd','Southampton','Tottenham','West Brom','West Ham','Wolves']
for i in range(380):
  for j in ['Home','Away']:
    for k in range(20):
      if results[j][i]==Name[k]:
        results[j][i]=k
results

,Home,HxG,HScore,AScore,AxG,Away
0,7,0.2,0.0,3.0,1.8,0
1,5,0.7,1.0,0.0,0.8,15
2,10,3.3,4.0,3.0,0.6,8
3,18,1.1,0.0,2.0,1.5,13
4,17,0.5,0.0,3.0,2.2,9
...,...,...,...,...,...,...
375,0,NaN,NaN,NaN,NaN,2
376,18,NaN,NaN,NaN,NaN,15
377,14,NaN,NaN,NaN,NaN,3
378,10,NaN,NaN,NaN,NaN,5


In [ ]:
traindata=results.iloc[0:x,:]
preddata=results.iloc[x:380,:]
newresults=results
x_train = traindata[['Home', 'Away']]
y_train = traindata[['HScore', 'AScore']]
x_pred = preddata[['Home', 'Away']]
newresults

,Home,HxG,HScore,AScore,AxG,Away
0,7,0.2,0.0,3.0,1.8,0
1,5,0.7,1.0,0.0,0.8,15
2,10,3.3,4.0,3.0,0.6,8
3,18,1.1,0.0,2.0,1.5,13
4,17,0.5,0.0,3.0,2.2,9
...,...,...,...,...,...,...
375,0,NaN,NaN,NaN,NaN,2
376,18,NaN,NaN,NaN,NaN,15
377,14,NaN,NaN,NaN,NaN,3
378,10,NaN,NaN,NaN,NaN,5


In [ ]:
model = MultiOutputRegressor(KNeighborsRegressor(n_neighbors=3, weights='distance')).fit(x_train, y_train)

In [ ]:
goals = model.predict(x_pred)

In [ ]:
for i in range(380-x):
  for j in range(1):
    if j==0:
      k='HScore'
    if j==1:
      k='AScore'
    newresults[k][x+i]=goals[i][j]
newresults

,Home,HxG,HScore,AScore,AxG,Away
0,7,0.2,0.000000,3.0,1.8,0
1,5,0.7,1.000000,0.0,0.8,15
2,10,3.3,4.000000,3.0,0.6,8
3,18,1.1,0.000000,2.0,1.5,13
4,17,0.5,0.000000,3.0,2.2,9
...,...,...,...,...,...,...
375,0,NaN,0.333333,NaN,NaN,2
376,18,NaN,0.707107,NaN,NaN,15
377,14,NaN,1.242641,NaN,NaN,3
378,10,NaN,2.242641,NaN,NaN,5


In [ ]:
lb = {'Team': Name,
    'Win':np.zeros((20)),
    'Loss':np.zeros((20)),
    'Draw':np.zeros((20)),
    'Pts':np.zeros((20))
      }
leaderboard = pd.DataFrame(lb, columns= ['Team','Win','Loss','Draw','Pts'])
for i in range(380):
  if newresults['HScore'][i]>newresults['AScore'][i]:
    leaderboard['Win'][newresults['Home'][i]]+=1
    leaderboard['Pts'][newresults['Home'][i]]+=3
    leaderboard['Loss'][newresults['Away'][i]]+=1
    leaderboard['Pts'][newresults['Away'][i]]+=0
  if newresults['HScore'][i]==newresults['AScore'][i]:
    leaderboard['Draw'][newresults['Home'][i]]+=1
    leaderboard['Pts'][newresults['Home'][i]]+=1
    leaderboard['Draw'][newresults['Away'][i]]+=1
    leaderboard['Pts'][newresults['Away'][i]]+=1
  if newresults['HScore'][i]<newresults['AScore'][i]:
    leaderboard['Loss'][newresults['Home'][i]]+=1
    leaderboard['Pts'][newresults['Home'][i]]+=0
    leaderboard['Win'][newresults['Away'][i]]+=1
    leaderboard['Pts'][newresults['Away'][i]]+=3

In [ ]:
leaderboard.sort_values(by=['Pts','Win'],inplace=True, ascending=False)
leaderboard

,Team,Win,Loss,Draw,Pts
11,Manchester City,21.0,5.0,5.0,68.0
12,Manchester Utd,17.0,8.0,5.0,56.0
10,Liverpool,16.0,6.0,8.0,56.0
16,Tottenham,15.0,8.0,7.0,52.0
0,Arsenal,15.0,11.0,4.0,49.0
6,Everton,15.0,10.0,4.0,49.0
8,Leeds United,15.0,12.0,3.0,48.0
9,Leicester City,15.0,12.0,3.0,48.0
18,West Ham,14.0,10.0,6.0,48.0
4,Chelsea,14.0,11.0,5.0,47.0


Part2: Using xG to predict the goal efficiency of each team in each match

In [ ]:
#Change name to ID
results2 = data.iloc[:,1:7]
Name=['Arsenal','Aston Villa','Brighton','Burnley','Chelsea','Crystal Palace','Everton','Fulham','Leeds United','Leicester City','Liverpool','Manchester City','Manchester Utd','Newcastle Utd','Sheffield Utd','Southampton','Tottenham','West Brom','West Ham','Wolves']
for i in range(380):
  for j in ['Home','Away']:
    for k in range(20):
      if results2[j][i]==Name[k]:
        results2[j][i]=k
results2.insert(1,'HGoalEff',np.zeros(380), True)
results2.insert(6,'AGoalEff',np.zeros(380), True)
results2

,Home,HGoalEff,HxG,HScore,AScore,AxG,AGoalEff,Away
0,7,0.0,0.2,0.0,3.0,1.8,0.0,0
1,5,0.0,0.7,1.0,0.0,0.8,0.0,15
2,10,0.0,3.3,4.0,3.0,0.6,0.0,8
3,18,0.0,1.1,0.0,2.0,1.5,0.0,13
4,17,0.0,0.5,0.0,3.0,2.2,0.0,9
...,...,...,...,...,...,...,...,...
375,0,0.0,NaN,NaN,NaN,NaN,0.0,2
376,18,0.0,NaN,NaN,NaN,NaN,0.0,15
377,14,0.0,NaN,NaN,NaN,NaN,0.0,3
378,10,0.0,NaN,NaN,NaN,NaN,0.0,5


In [ ]:
GoalEff=[np.arange(20),np.zeros(20),np.zeros(20),np.zeros(20),np.zeros(20)]
for i in range(x):
  if results2['HScore'][i]==0:
    GoalEff[1][results2['Home'][i]]+=(results2['HScore'][i]-results2['HxG'][i])/0.5
  else:
    GoalEff[1][results2['Home'][i]]+=(results2['HScore'][i]-results2['HxG'][i])/results2['HScore'][i]
  GoalEff[2][results2['Home'][i]]+=1
  if results2['AScore'][i]==0:
    GoalEff[1][results2['Away'][i]]+=(results2['AScore'][i]-results2['AxG'][i])/0.5
  else:
    GoalEff[1][results2['Away'][i]]+=(results2['AScore'][i]-results2['AxG'][i])/results2['AScore'][i]
  GoalEff[2][results2['Away'][i]]+=1
for i in range(20):
  GoalEff[3][i]=GoalEff[1][i]/GoalEff[2][i]
for i in range(x):
  if results2['HScore'][i]==0:
    GoalEff[4][results2['Home'][i]]+=((results2['HScore'][i]-results2['HxG'][i])/0.5-GoalEff[3][results2['Home'][i]])**2
  else:
    GoalEff[4][results2['Home'][i]]+=((results2['HScore'][i]-results2['HxG'][i])/results2['HScore'][i]-GoalEff[3][results2['Home'][i]])**2
  if results2['AScore'][i]==0:
    GoalEff[4][results2['Away'][i]]+=((results2['AScore'][i]-results2['AxG'][i])/0.5-GoalEff[3][results2['Away'][i]])**2
  else:
    GoalEff[4][results2['Away'][i]]+=((results2['AScore'][i]-results2['AxG'][i])/results2['AScore'][i]-GoalEff[3][results2['Away'][i]])**2
for i in range(20):
  GoalEff[4][i]=GoalEff[4][i]/GoalEff[2][i]


In [ ]:
for i in range(x):
  if results2['HScore'][i]==0:
    results2['HGoalEff'][i]=(results2['HScore'][i]-results2['HxG'][i])/0.5
  else:
    results2['HGoalEff'][i]=(results2['HScore'][i]-results2['HxG'][i])/results2['HScore'][i]
  if results2['AScore'][i]==0:
    results2['AGoalEff'][i]=(results2['AScore'][i]-results2['AxG'][i])/0.5
  else:
    results2['AGoalEff'][i]=(results2['AScore'][i]-results2['AxG'][i])/results2['AScore'][i]
for i in range(380-x):
  results2['HGoalEff'][x+i]=np.random.normal(GoalEff[3][results2['Home'][x+i]],GoalEff[4][results2['Home'][x+i]])
  results2['AGoalEff'][x+i]=np.random.normal(GoalEff[3][results2['Away'][x+i]],GoalEff[4][results2['Away'][x+i]])
results2

,Home,HGoalEff,HxG,HScore,AScore,AxG,AGoalEff,Away
0,7,-0.400000,0.2,0.0,3.0,1.8,0.400000,0
1,5,0.300000,0.7,1.0,0.0,0.8,-1.600000,15
2,10,0.175000,3.3,4.0,3.0,0.6,0.800000,8
3,18,-2.200000,1.1,0.0,2.0,1.5,0.250000,13
4,17,-1.000000,0.5,0.0,3.0,2.2,0.266667,9
...,...,...,...,...,...,...,...,...
375,0,-1.955635,NaN,NaN,NaN,NaN,-0.844115,2
376,18,-1.402910,NaN,NaN,NaN,NaN,-0.981102,15
377,14,-3.127062,NaN,NaN,NaN,NaN,-1.681089,3
378,10,-0.709130,NaN,NaN,NaN,NaN,-0.168639,5


In [ ]:
traindata2=results2.iloc[0:x,:]
preddata2=results2.iloc[x:380,:]
newresults2=results2
x_train2 = traindata2[['Home','HGoalEff','AGoalEff','Away']]
y_train2 = traindata2[['HScore', 'AScore']]
x_pred2 = preddata2[['Home','HGoalEff','AGoalEff','Away']]

In [ ]:
model2 = MultiOutputRegressor(KNeighborsRegressor(n_neighbors=3, weights='distance')).fit(x_train2, y_train2)

In [ ]:
goals2 = model2.predict(x_pred2)

In [ ]:
for i in range(380-x):
  for j in range(2):
    if j==0:
      k='HScore'
    if j==1:
      k='AScore'
    newresults2[k][x+i]=goals2[i][j]
newresults2

In [ ]:
lb = {'Team': Name,
    'Win':np.zeros((20)),
    'Loss':np.zeros((20)),
    'Draw':np.zeros((20)),
    'Pts':np.zeros((20))
      }
leaderboard2 = pd.DataFrame(lb, columns= ['Team','Win','Loss','Draw','Pts'])
for i in range(380):
  if newresults2['HScore'][i]>newresults2['AScore'][i]:
    leaderboard2['Win'][newresults2['Home'][i]]+=1
    leaderboard2['Pts'][newresults2['Home'][i]]+=3
    leaderboard2['Loss'][newresults2['Away'][i]]+=1
    leaderboard2['Pts'][newresults2['Away'][i]]+=0
  if newresults2['HScore'][i]==newresults2['AScore'][i]:
    leaderboard2['Draw'][newresults2['Home'][i]]+=1
    leaderboard2['Pts'][newresults2['Home'][i]]+=1
    leaderboard2['Draw'][newresults2['Away'][i]]+=1
    leaderboard2['Pts'][newresults2['Away'][i]]+=1
  if newresults2['HScore'][i]<newresults2['AScore'][i]:
    leaderboard2['Loss'][newresults2['Home'][i]]+=1
    leaderboard2['Pts'][newresults2['Home'][i]]+=0
    leaderboard2['Win'][newresults2['Away'][i]]+=1
    leaderboard2['Pts'][newresults2['Away'][i]]+=3

In [ ]:
leaderboard2.sort_values(by=['Pts','Win'],inplace=True, ascending=False)
leaderboard2

,Team,Win,Loss,Draw,Pts
12,Manchester Utd,24.0,10.0,4.0,76.0
11,Manchester City,23.0,8.0,7.0,76.0
9,Leicester City,22.0,12.0,4.0,70.0
15,Southampton,21.0,12.0,5.0,68.0
10,Liverpool,20.0,10.0,8.0,68.0
0,Arsenal,21.0,14.0,3.0,66.0
6,Everton,21.0,14.0,3.0,66.0
4,Chelsea,20.0,13.0,5.0,65.0
16,Tottenham,18.0,14.0,6.0,60.0
1,Aston Villa,17.0,19.0,2.0,53.0


Repeat Part 2 1000 times to find the expected win percentage

In [ ]:
y=1000
results2 = data.iloc[:,1:7]
Name=['Arsenal','Aston Villa','Brighton','Burnley','Chelsea','Crystal Palace','Everton','Fulham','Leeds United','Leicester City','Liverpool','Manchester City','Manchester Utd','Newcastle Utd','Sheffield Utd','Southampton','Tottenham','West Brom','West Ham','Wolves']
for i in range(380):
  for j in ['Home','Away']:
    for k in range(20):
      if results2[j][i]==Name[k]:
        results2[j][i]=k
results2.insert(1,'HGoalEff',np.zeros(380), True)
results2.insert(6,'AGoalEff',np.zeros(380), True)
GoalEff=[np.arange(20),np.zeros(20),np.zeros(20),np.zeros(20),np.zeros(20)]  
aclb = {'Team': Name,
  'Champ':np.zeros(20),
  'Champ%':np.zeros(20),
  'Top4':np.zeros(20),
  'Top4%':np.zeros(20),
  'Rel':np.zeros(20),
  'Rel%':np.zeros(20)
    }  
for i in range(x):
  if results2['HScore'][i]==0:
    GoalEff[1][results2['Home'][i]]+=(results2['HScore'][i]-results2['HxG'][i])/0.5
  else:
    GoalEff[1][results2['Home'][i]]+=(results2['HScore'][i]-results2['HxG'][i])/results2['HScore'][i]
  GoalEff[2][results2['Home'][i]]+=1
  if results2['AScore'][i]==0:
    GoalEff[1][results2['Away'][i]]+=(results2['AScore'][i]-results2['AxG'][i])/0.5
  else:
    GoalEff[1][results2['Away'][i]]+=(results2['AScore'][i]-results2['AxG'][i])/results2['AScore'][i]
  GoalEff[2][results2['Away'][i]]+=1
for i in range(20):
  GoalEff[3][i]=GoalEff[1][i]/GoalEff[2][i]
for i in range(x):
  if results2['HScore'][i]==0:
    GoalEff[4][results2['Home'][i]]+=((results2['HScore'][i]-results2['HxG'][i])/0.5-GoalEff[3][results2['Home'][i]])**2
  else:
    GoalEff[4][results2['Home'][i]]+=((results2['HScore'][i]-results2['HxG'][i])/results2['HScore'][i]-GoalEff[3][results2['Home'][i]])**2
  if results2['AScore'][i]==0:
    GoalEff[4][results2['Away'][i]]+=((results2['AScore'][i]-results2['AxG'][i])/0.5-GoalEff[3][results2['Away'][i]])**2
  else:
    GoalEff[4][results2['Away'][i]]+=((results2['AScore'][i]-results2['AxG'][i])/results2['AScore'][i]-GoalEff[3][results2['Away'][i]])**2
for i in range(20):
  GoalEff[4][i]=GoalEff[4][i]/GoalEff[2][i]
for i in range(x):
  if results2['HScore'][i]==0:
    results2['HGoalEff'][i]=(results2['HScore'][i]-results2['HxG'][i])/0.5
  else:
    results2['HGoalEff'][i]=(results2['HScore'][i]-results2['HxG'][i])/results2['HScore'][i]
  if results2['AScore'][i]==0:
    results2['AGoalEff'][i]=(results2['AScore'][i]-results2['AxG'][i])/0.5
  else:
    results2['AGoalEff'][i]=(results2['AScore'][i]-results2['AxG'][i])/results2['AScore'][i]
for a in range(y):
    for i in range(380-x):
      results2['HGoalEff'][x+i]=np.random.normal(GoalEff[3][results2['Home'][x+i]],GoalEff[4][results2['Home'][x+i]])
      results2['AGoalEff'][x+i]=np.random.normal(GoalEff[3][results2['Away'][x+i]],GoalEff[4][results2['Away'][x+i]])
    traindata2=results2.iloc[0:x,:]
    preddata2=results2.iloc[x:380,:]
    newresults2=results2
    x_train2 = traindata2[['Home','HGoalEff','AGoalEff','Away']]
    y_train2 = traindata2[['HScore', 'AScore']]
    x_pred2 = preddata2[['Home','HGoalEff','AGoalEff','Away']]
    model2 = MultiOutputRegressor(KNeighborsRegressor(n_neighbors=3, weights='distance')).fit(x_train2, y_train2)
    goals2 = model2.predict(x_pred2)
    for i in range(380-x):
      for j in range(2):
        if j==0:
          k='HScore'
        if j==1:
          k='AScore'
        newresults2[k][x+i]=goals2[i][j]
    lb = {'Team': np.arange(20),
    'Win':np.zeros((20)),
    'Loss':np.zeros((20)),
    'Draw':np.zeros((20)),
    'Pts':np.zeros((20))
      }
    leaderboard2 = pd.DataFrame(lb, columns= ['Team','Win','Loss','Draw','Pts']) 
    for i in range(380):
      if newresults2['HScore'][i]>newresults2['AScore'][i]:
        leaderboard2['Win'][newresults2['Home'][i]]+=1
        leaderboard2['Pts'][newresults2['Home'][i]]+=3
        leaderboard2['Loss'][newresults2['Away'][i]]+=1
        leaderboard2['Pts'][newresults2['Away'][i]]+=0
      if newresults2['HScore'][i]==newresults2['AScore'][i]:
        leaderboard2['Draw'][newresults2['Home'][i]]+=1
        leaderboard2['Pts'][newresults2['Home'][i]]+=1
        leaderboard2['Draw'][newresults2['Away'][i]]+=1
        leaderboard2['Pts'][newresults2['Away'][i]]+=1
      if newresults2['HScore'][i]<newresults2['AScore'][i]:
        leaderboard2['Loss'][newresults2['Home'][i]]+=1
        leaderboard2['Pts'][newresults2['Home'][i]]+=0
        leaderboard2['Win'][newresults2['Away'][i]]+=1
        leaderboard2['Pts'][newresults2['Away'][i]]+=3
    c=leaderboard2.loc[leaderboard2['Pts'].idxmax(), 'Team']
    aclb['Champ'][c]+=1
    for l in range(4):
      d=leaderboard2.nlargest(4,'Pts')['Team'].iloc[l]
      aclb['Top4'][d]+=1
    for l in range(3):
      d=leaderboard2.nsmallest(3,'Pts')['Team'].iloc[l]
      aclb['Rel'][d]+=1    
accleaderboard = pd.DataFrame(aclb, columns= ['Team','Champ','Win%','Top4','Top4%','Rel','Rel%'])
accleaderboard['Win%']=accleaderboard['Champ']/y*100
accleaderboard['Top4%']=accleaderboard['Top4']/y*100
accleaderboard['Rel%']=accleaderboard['Rel']/y*100
accleaderboard.sort_values(by=['Rel'],inplace=True, ascending=True)
accleaderboard.sort_values(by=['Champ','Top4'],inplace=True, ascending=False)
accleaderboard

,Team,Champ,Win%,Top4,Top4%,Rel,Rel%
11,Manchester City,623.0,62.3,990.0,99.0,0.0,0.0
12,Manchester Utd,185.0,18.5,904.0,90.4,0.0,0.0
10,Liverpool,141.0,14.1,779.0,77.9,0.0,0.0
9,Leicester City,42.0,4.2,593.0,59.3,0.0,0.0
6,Everton,7.0,0.7,238.0,23.8,0.0,0.0
0,Arsenal,1.0,0.1,180.0,18.0,0.0,0.0
16,Tottenham,1.0,0.1,98.0,9.8,0.0,0.0
4,Chelsea,0.0,0.0,97.0,9.7,0.0,0.0
1,Aston Villa,0.0,0.0,58.0,5.8,1.0,0.1
15,Southampton,0.0,0.0,51.0,5.1,0.0,0.0


Def eff

In [14]:
y=1000
results2 = data.iloc[:,1:7]
Name=['Arsenal','Aston Villa','Brighton','Burnley','Chelsea','Crystal Palace','Everton','Fulham','Leeds United','Leicester City','Liverpool','Manchester City','Manchester Utd','Newcastle Utd','Sheffield Utd','Southampton','Tottenham','West Brom','West Ham','Wolves']
for i in range(380):
  for j in ['Home','Away']:
    for k in range(20):
      if results2[j][i]==Name[k]:
        results2[j][i]=k
results2.insert(1,'HGoalEff',np.zeros(380), True)
results2.insert(2,'HDefEff',np.zeros(380), True)
results2.insert(6,'AGoalEff',np.zeros(380), True)
results2.insert(6,'ADefEff',np.zeros(380), True)
GoalEff=[np.arange(20),np.zeros(20),np.zeros(20),np.zeros(20),np.zeros(20)]
DefEff=[np.arange(20),np.zeros(20),np.zeros(20),np.zeros(20),np.zeros(20)]   
aclb = {'Team': Name,
  'Champ':np.zeros(20),
  'Champ%':np.zeros(20),
  'Top4':np.zeros(20),
  'Top4%':np.zeros(20),
  'Rel':np.zeros(20),
  'Rel%':np.zeros(20)
    }  
for i in range(x):
  if results2['HScore'][i]==0:
    GoalEff[1][results2['Home'][i]]+=(results2['HScore'][i]-results2['HxG'][i])/0.5
  else:
    GoalEff[1][results2['Home'][i]]+=(results2['HScore'][i]-results2['HxG'][i])/results2['HScore'][i]
  GoalEff[2][results2['Home'][i]]+=1
  if results2['AScore'][i]==0:
    GoalEff[1][results2['Away'][i]]+=(results2['AScore'][i]-results2['AxG'][i])/0.5
  else:
    GoalEff[1][results2['Away'][i]]+=(results2['AScore'][i]-results2['AxG'][i])/results2['AScore'][i]
  GoalEff[2][results2['Away'][i]]+=1
for i in range(20):
  GoalEff[3][i]=GoalEff[1][i]/GoalEff[2][i]
for i in range(x):
  if results2['HScore'][i]==0:
    GoalEff[4][results2['Home'][i]]+=((results2['HScore'][i]-results2['HxG'][i])/0.5-GoalEff[3][results2['Home'][i]])**2
  else:
    GoalEff[4][results2['Home'][i]]+=((results2['HScore'][i]-results2['HxG'][i])/results2['HScore'][i]-GoalEff[3][results2['Home'][i]])**2
  if results2['AScore'][i]==0:
    GoalEff[4][results2['Away'][i]]+=((results2['AScore'][i]-results2['AxG'][i])/0.5-GoalEff[3][results2['Away'][i]])**2
  else:
    GoalEff[4][results2['Away'][i]]+=((results2['AScore'][i]-results2['AxG'][i])/results2['AScore'][i]-GoalEff[3][results2['Away'][i]])**2
for i in range(20):
  GoalEff[4][i]=GoalEff[4][i]/GoalEff[2][i]
for i in range(x):
  if results2['HScore'][i]==0:
    results2['HGoalEff'][i]=(results2['HScore'][i]-results2['HxG'][i])/0.5
  else:
    results2['HGoalEff'][i]=(results2['HScore'][i]-results2['HxG'][i])/results2['HScore'][i]
  if results2['AScore'][i]==0:
    results2['AGoalEff'][i]=(results2['AScore'][i]-results2['AxG'][i])/0.5
  else:
    results2['AGoalEff'][i]=(results2['AScore'][i]-results2['AxG'][i])/results2['AScore'][i]
for i in range(x):#Def
  if results2['AScore'][i]==0:
    DefEff[1][results2['Home'][i]]+=(results2['AxG'][i]-results2['AScore'][i])/0.5
  else:
    DefEff[1][results2['Home'][i]]+=(results2['AxG'][i]-results2['AScore'][i])/results2['AScore'][i]
  DefEff[2][results2['Home'][i]]+=1
  if results2['HScore'][i]==0:
    DefEff[1][results2['Away'][i]]+=(results2['HxG'][i]-results2['HScore'][i])/0.5
  else:
    DefEff[1][results2['Away'][i]]+=(results2['HxG'][i]-results2['HScore'][i])/results2['HScore'][i]
  DefEff[2][results2['Away'][i]]+=1
for i in range(20):
  DefEff[3][i]=DefEff[1][i]/DefEff[2][i]
for i in range(x):
  if results2['AScore'][i]==0:
    DefEff[4][results2['Away'][i]]+=((results2['AxG'][i]-results2['AScore'][i])/0.5-DefEff[3][results2['Away'][i]])**2
  else:
    DefEff[4][results2['Home'][i]]+=((results2['AxG'][i]-results2['AScore'][i])/results2['AScore'][i]-DefEff[3][results2['Away'][i]])**2
  if results2['HScore'][i]==0:
    DefEff[4][results2['Away'][i]]+=((results2['HScore'][i]-results2['HxG'][i])/0.5-DefEff[3][results2['Home'][i]])**2
  else:
    DefEff[4][results2['Away'][i]]+=((results2['HScore'][i]-results2['HxG'][i])/results2['HScore'][i]-DefEff[3][results2['Home'][i]])**2
for i in range(20):
  DefEff[4][i]=DefEff[4][i]/DefEff[2][i]
for i in range(x):
  if results2['AScore'][i]==0:
    results2['HDefEff'][i]=0.5*(results2['AScore'][i]-results2['AxG'][i])/0.5
  else:
    results2['HDefEff'][i]=0.5*(results2['AScore'][i]-results2['AxG'][i])/results2['AScore'][i]
  if results2['HScore'][i]==0:
    results2['ADefEff'][i]=0.5*(results2['HScore'][i]-results2['HxG'][i])/0.5
  else:
    results2['ADefEff'][i]=0.5*(results2['HScore'][i]-results2['HxG'][i])/results2['HScore'][i]
for a in range(y):#model
    for i in range(380-x):
      results2['HGoalEff'][x+i]=np.random.normal(GoalEff[3][results2['Home'][x+i]],GoalEff[4][results2['Home'][x+i]])
      results2['AGoalEff'][x+i]=np.random.normal(GoalEff[3][results2['Away'][x+i]],GoalEff[4][results2['Away'][x+i]])
      results2['HDefEff'][x+i]=np.random.normal(DefEff[3][results2['Home'][x+i]],DefEff[4][results2['Home'][x+i]]/2)/2
      results2['ADefEff'][x+i]=np.random.normal(DefEff[3][results2['Away'][x+i]],DefEff[4][results2['Away'][x+i]]/2)/2
    traindata2=results2.iloc[0:x,:]
    preddata2=results2.iloc[x:380,:]
    newresults2=results2
    x_train2 = traindata2[['Home','HGoalEff','HDefEff','AGoalEff','ADefEff','Away']]
    y_train2 = traindata2[['HScore', 'AScore']]
    x_pred2 = preddata2[['Home','HGoalEff','HDefEff','AGoalEff','ADefEff','Away']]
    model2 = MultiOutputRegressor(KNeighborsRegressor(n_neighbors=3, weights='distance')).fit(x_train2, y_train2)
    goals2 = model2.predict(x_pred2)
    for i in range(380-x):
      for j in range(2):
        if j==0:
          k='HScore'
        if j==1:
          k='AScore'
        newresults2[k][x+i]=goals2[i][j]
    lb = {'Team': np.arange(20),
    'Win':np.zeros((20)),
    'Loss':np.zeros((20)),
    'Draw':np.zeros((20)),
    'Pts':np.zeros((20))
      }
    leaderboard2 = pd.DataFrame(lb, columns= ['Team','Win','Loss','Draw','Pts']) 
    for i in range(380):
      if newresults2['HScore'][i]>newresults2['AScore'][i]:
        leaderboard2['Win'][newresults2['Home'][i]]+=1
        leaderboard2['Pts'][newresults2['Home'][i]]+=3
        leaderboard2['Loss'][newresults2['Away'][i]]+=1
        leaderboard2['Pts'][newresults2['Away'][i]]+=0
      if newresults2['HScore'][i]==newresults2['AScore'][i]:
        leaderboard2['Draw'][newresults2['Home'][i]]+=1
        leaderboard2['Pts'][newresults2['Home'][i]]+=1
        leaderboard2['Draw'][newresults2['Away'][i]]+=1
        leaderboard2['Pts'][newresults2['Away'][i]]+=1
      if newresults2['HScore'][i]<newresults2['AScore'][i]:
        leaderboard2['Loss'][newresults2['Home'][i]]+=1
        leaderboard2['Pts'][newresults2['Home'][i]]+=0
        leaderboard2['Win'][newresults2['Away'][i]]+=1
        leaderboard2['Pts'][newresults2['Away'][i]]+=3
    c=leaderboard2.loc[leaderboard2['Pts'].idxmax(), 'Team']
    aclb['Champ'][c]+=1
    for l in range(4):
      d=leaderboard2.nlargest(4,'Pts')['Team'].iloc[l]
      aclb['Top4'][d]+=1
    for l in range(3):
      d=leaderboard2.nsmallest(3,'Pts')['Team'].iloc[l]
      aclb['Rel'][d]+=1    
accleaderboard = pd.DataFrame(aclb, columns= ['Team','Champ','Win%','Top4','Top4%','Rel','Rel%'])
accleaderboard['Win%']=accleaderboard['Champ']/y*100
accleaderboard['Top4%']=accleaderboard['Top4']/y*100
accleaderboard['Rel%']=accleaderboard['Rel']/y*100
accleaderboard.sort_values(by=['Rel'],inplace=True, ascending=True)
accleaderboard.sort_values(by=['Champ','Top4'],inplace=True, ascending=False)
accleaderboard

,Team,Champ,Win%,Top4,Top4%,Rel,Rel%
11,Manchester City,554.0,55.4,985.0,98.5,0.0,0.0
10,Liverpool,194.0,19.4,810.0,81.0,0.0,0.0
12,Manchester Utd,178.0,17.8,870.0,87.0,0.0,0.0
9,Leicester City,44.0,4.4,508.0,50.8,0.0,0.0
6,Everton,18.0,1.8,330.0,33.0,0.0,0.0
0,Arsenal,6.0,0.6,203.0,20.3,0.0,0.0
4,Chelsea,6.0,0.6,130.0,13.0,0.0,0.0
16,Tottenham,0.0,0.0,67.0,6.7,0.0,0.0
1,Aston Villa,0.0,0.0,39.0,3.9,0.0,0.0
15,Southampton,0.0,0.0,38.0,3.8,0.0,0.0


In [15]:
DefEff

[array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19]),
 array([10.88333333, 12.69166667,  1.1       , 20.09333333,  4.71666667,
         1.81666667,  7.03333333,  5.09166667, 12.39166667,  7.76666667,
         9.20714286,  9.97      ,  8.71666667,  3.95166667,  1.65833333,
         9.35      ,  9.3       ,  6.22833333,  7.73333333,  4.76666667]),
 array([20., 18., 19., 19., 19., 20., 17., 18., 19., 19., 19., 19., 19.,
        20., 19., 19., 18., 20., 20., 19.]),
 array([0.54416667, 0.70509259, 0.05789474, 1.05754386, 0.24824561,
        0.09083333, 0.41372549, 0.28287037, 0.65219298, 0.40877193,
        0.48458647, 0.52473684, 0.45877193, 0.19758333, 0.0872807 ,
        0.49210526, 0.51666667, 0.31141667, 0.38666667, 0.25087719]),
 array([2.45016394, 1.73837059, 1.24952455, 4.30573415, 1.05957741,
        1.30731168, 1.25744688, 1.10348478, 2.16038148, 1.33459775,
        1.03551431, 1.38111856, 1.5691619 , 1.24708338, 2.20651424,
        0.87

In [8]:
pd.DataFrame(GoalEff,index=['Team','Total','Num','Mean','Var'])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
Team,0.000000,1.000000,2.000000,3.000000,4.000000,5.000000,6.000000,7.000000,8.000000,9.000000,10.000000,11.000000,12.000000,13.000000,14.000000,15.000000,16.000000,17.000000,18.000000,19.000000
Total,-12.625000,-8.340476,-9.850000,-11.516667,-7.400000,-6.268333,-2.113333,-11.500000,-12.565000,-3.845000,-12.775000,-8.418333,-4.758333,-4.116667,-17.700000,-1.675000,-3.700000,-5.550000,-5.033333,-4.716667
Num,20.000000,18.000000,19.000000,19.000000,19.000000,20.000000,17.000000,18.000000,19.000000,19.000000,19.000000,19.000000,19.000000,20.000000,19.000000,19.000000,18.000000,20.000000,20.000000,19.000000
Mean,-0.631250,-0.463360,-0.518421,-0.606140,-0.389474,-0.313417,-0.124314,-0.638889,-0.661316,-0.202368,-0.672368,-0.443070,-0.250439,-0.205833,-0.931579,-0.088158,-0.205556,-0.277500,-0.251667,-0.248246
Var,1.262832,1.371764,0.906927,0.691380,0.766592,1.147795,0.809557,0.928210,2.071694,0.671542,1.958027,0.868619,0.635603,0.428313,2.136898,0.708343,0.810926,0.699563,0.730303,0.566020
